In [ ]:
import twitterGraph as tg
import numpy as np
import random as rand
import tweepy
from config import tw_key, tw_secret
graph = tg.twitterGraph()
auth = auth = tweepy.AppAuthHandler(tw_key, tw_secret)
api = tweepy.API(auth)

In [5]:
lists = api.lists_all("CSPAN")

In [ ]:
import json
with 

In [38]:
[x.name for x in lists]

['Impeachment',
 'Select Cmte Leaders',
 'House Leadership',
 'Senate Leadership',
 'POTUS',
 'House Intelligence Cmte',
 'House Judiciary Cmte',
 'More from C-SPAN staff',
 'Senate Judiciary Cmte',
 'The Cabinet',
 'New Members of Congress',
 'Political Reporters',
 'Presidential Candidates',
 'Foreign Leaders',
 'Supreme Court Reporters',
 'Congressional Committees',
 'Members of Congress',
 'Congressional Media',
 'Governors',
 'U.S. Representatives',
 'Senators',
 'C-SPAN Twitter feeds']

In [26]:
lists[16].__dir__()

['_api',
 '_json',
 'id',
 'id_str',
 'name',
 'uri',
 'subscriber_count',
 'member_count',
 'mode',
 'description',
 'slug',
 'full_name',
 'created_at',
 'following',
 'user',
 '__module__',
 'parse',
 'parse_list',
 'update',
 'destroy',
 'timeline',
 'add_member',
 'remove_member',
 'members',
 'is_member',
 'subscribe',
 'unsubscribe',
 'subscribers',
 'is_subscribed',
 '__doc__',
 '__init__',
 '__getstate__',
 '__repr__',
 '__dict__',
 '__weakref__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [37]:
for item in tweepy.Cursor(api.list_members, list_id=lists[18].id).items():
    politicians.append(item)

In [39]:
len(politicians)

611

In [40]:
politicians[0].__dir__()

['_api',
 '_json',
 'id',
 'id_str',
 'name',
 'screen_name',
 'location',
 'description',
 'url',
 'entities',
 'protected',
 'followers_count',
 'friends_count',
 'listed_count',
 'created_at',
 'favourites_count',
 'utc_offset',
 'time_zone',
 'geo_enabled',
 'verified',
 'statuses_count',
 'lang',
 'status',
 'contributors_enabled',
 'is_translator',
 'is_translation_enabled',
 'profile_background_color',
 'profile_background_image_url',
 'profile_background_image_url_https',
 'profile_background_tile',
 'profile_image_url',
 'profile_image_url_https',
 'profile_banner_url',
 'profile_link_color',
 'profile_sidebar_border_color',
 'profile_sidebar_fill_color',
 'profile_text_color',
 'profile_use_background_image',
 'has_extended_profile',
 'default_profile',
 'default_profile_image',
 'following',
 'follow_request_sent',
 'notifications',
 'translator_type',
 '__module__',
 'parse',
 'parse_list',
 'timeline',
 'friends',
 'followers',
 'follow',
 'unfollow',
 'lists_memberships',

In [51]:
dat = pd.DataFrame({
    "id":[x.id for x in politicians],
    'name':[x.name for x in politicians],
    'screen_name':[x.screen_name for x in politicians],
    'location':[x.location for x in politicians],
    'desc':[x.description for x in politicians],
    'created_at':[x.created_at for x in politicians],
    'url':[x.url for x in politicians]
})

In [57]:
graph.getUser(30174586)

(30174586, 18, 4093808892, 56, 2703416108)

In [53]:
oldPoliticians = [graph.getUser(x.id) for x in politicians if graph.getUser(x.id)]

In [58]:
len(oldPoliticians)

121